In [1]:
cd '/home/psadmin/work/gradle/gradle-server-acceptance/cleansing'

/home/psadmin/work/gradle/gradle-server-acceptance/cleansing


In [3]:
import re
import os
import math
import logging
from enum import Enum
from datetime import datetime
from abc import ABCMeta, abstractmethod
import json
import numpy as np
import pandas as pd
import dataset as ds
from dataset.types import Types
from getconfig_cleansing.evidence.util import Util
from getconfig_cleansing.evidence.loader_v1 import GetconfigEvidenceV1
from getconfig_cleansing.evidence.inventory import InventoryInfo


In [5]:
excel = 'tests/resources/import/old1/build/check_sheet_20170512_143424.xlsx'
db = GetconfigEvidenceV1(excel, export_dir='build/tmp')
db.load()
len(db.df_summary)

Target
CheckSheet(Linux)
Index(['Test', 'ID', '項目', '分類', 'デバイス', '採取情報', 'ostrich'], dtype='object')
CheckSheet(Windows)
Index(['Test', 'ID', '項目', '分類', 'デバイス', '採取情報', 'win2012'], dtype='object')
CheckSheet(VMHost)
Index(['Test', 'ID', '項目', '分類', 'デバイス', '採取情報'], dtype='object')


/home/psadmin/work/gradle/gradle-server-acceptance/cleansing/getconfig_cleansing/evidence/util.py:90: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  cond1 = df[column].str.contains(suffix) & \


Rule
vCenter_vm_storage
Linux_network
Linux_filesystem
Linux_user
Linux_service
Linux_packages
Windows_driver
Windows_filesystem
Windows_user
Windows_service
Windows_network
Windows_firewall


108

In [153]:
df = db.df_summary
df = df[df['domain'] == 'Linux']
cond = (df['test_id'].isin(['lsb', 'uname', 'cpu_total', 'mem_total']))
df2 = df[['node_name', 'test_id', 'value']][cond]
df2 = df2.set_index(['node_name', 'test_id'])
df2 = df2.unstack(level=0).T.reset_index()
del df2['level_0']
df2 = df2.set_index('node_name')
df2

test_id,cpu_total,lsb,mem_total,uname
node_name,,,,
ostrich,1,[CentOS release 6.7 (Final)],1922252,Linux ostrich 2.6.32-573.12.1.el6.x86_64


In [172]:
df = db.df_summary
df = df[df['domain'] == 'Windows']
cond = (df['test_id'].isin(['os_caption', 'os_architecture', 'cpu_total', 'mem_total']))
df2 = df[['node_name', 'test_id', 'value']][cond]
df2 = df2.set_index(['node_name', 'test_id'])
# 以下のコードだと、unstack後のカラム定義がMultiindexになる。戻し方が不明なため、
# 回避策としてunstack + T + reset_indexでカラムを再定義する
df2 = df2.unstack(level=0).T.reset_index()
del df2['level_0']
df2 = df2.set_index('node_name')
df2.rename(columns={'os_caption': 'OS名', 'os_architecture': 'アーキテクチャ', 
                    'cpu_total':'CPU数', 'mem_total':'MEM容量'},
           inplace=True)
df2

test_id,CPU数,アーキテクチャ,OS名
node_name,,,
win2012,2,64 ビット,Microsoft Windows Server 2012 R2 Standard 評価版


In [173]:
df = db.df_devices['Windows_filesystem']
df3 = df[df['device_id']!='NaN']
df3['disk'] = df3['device_id'] + ':' + df3['size_gb'].astype(int).astype(str)
df3 = df3.pivot(index='node_name', columns='device_id', values='disk')
df3 = df3.apply(','.join, axis=1)
df2['disk'] = df3
df2

test_id,CPU数,アーキテクチャ,OS名,disk
node_name,,,,
win2012,2,64 ビット,Microsoft Windows Server 2012 R2 Standard 評価版,C::42


In [164]:
df2['disk'] = df3
df2

test_id,cpu_total,os_architecture,os_caption,disk
node_name,,,,
win2012,2,64 ビット,Microsoft Windows Server 2012 R2 Standard 評価版,C::42


In [64]:
# ディスク構成の読み込み
df = db.df_devices['Linux_network']
df3 = df[df['ip']!='NaN']
df3['net'] = df3['device'] + ':' + df3['ip']
df3 = df3.pivot(index='node_name', columns='device', values='net')
df3 = df3.apply(','.join, axis=1)
df3


Series([], dtype: float64)

In [70]:
df = db.df_devices['Windows_filesystem']
df3 = df[df['device_id']!='NaN']
df3['disk'] = df3['device_id'] + ':' + df3['size_gb'].astype(int).astype(str)
df3 = df3.pivot(index='node_name', columns='device_id', values='disk')
df3 = df3.apply(','.join, axis=1)
df3

node_name
win2012    C::42
dtype: object